# Financial Times Data fetch and save

This notebook downloads all the data from the reuters about a specific topic and saves it to the database


In [1]:
import bs4
from bs4 import BeautifulSoup
import requests
import ray
from math import ceil
import time
from datetime import datetime
from http.client import HTTPSConnection
from urllib.parse import urljoin
import asyncio
import aiohttp
from time import sleep
import os.path
from dateutil import parser

In [2]:
num_cpus = 4

ray.init(ignore_reinit_error=True, num_cpus=num_cpus)

2024-06-21 11:31:50,506	INFO worker.py:1715 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 


Python version:,3.8.18
Ray version:,2.9.3
Dashboard:,http://127.0.0.1:8265


In [3]:
import duckdb

class NewsMetaRepository:
    def __init__(self, csv_file=None):
        self.connection = duckdb.connect(database=':memory:', read_only=False)
        if os.path.isfile(f'{csv_file}/news_meta.csv'):
            try:
                self.connection.execute(f"IMPORT DATABASE '{csv_file}';")
            except Exception as e:
                self._create_table()
                self.connection.execute(f"EXPORT DATABASE '{csv_file}';")
        else:
            self._create_table()

    def _create_table(self):
        self.connection.execute("CREATE TABLE news_meta (id VARCHAR, title VARCHAR UNIQUE, url VARCHAR, timestamp VARCHAR UNIQUE,term VARCHAR)")
        self.connection.execute("CREATE SEQUENCE id_sequence START 1 INCREMENT BY 1;")

    def insert(self, news_meta):
        self.connection.execute("PREPARE insert_meta AS "
                     "INSERT INTO news_meta VALUES (nextval('id_sequence'), ?, ?, ?, ?) ON CONFLICT DO NOTHING;")
        self.connection.execute(f"EXECUTE insert_meta('{news_meta['title']}', '{news_meta['url']}', '{news_meta['timestamp']}', '{news_meta['term']}');")

    def select_all(self):
        return self.connection.execute("SELECT * FROM news_meta").fetchdf()

    def select_by_id(self, id):
        return self.connection.execute("SELECT * FROM news_meta WHERE id = ?", id).fetchdf()

    def select_by_title(self, title):
        return self.connection.execute("SELECT * FROM news_meta WHERE title = ?", title).fetchdf()

    def select_by_url(self, url):
        return self.connection.execute("SELECT * FROM news_meta WHERE url = ?", url).fetchdf()

    def select_by_term(self, source):
        self.connection.execute("PREPARE select_by_source AS "
                                "SELECT * FROM news_meta WHERE term = ?")
        return self.connection.execute(f"EXECUTE select_by_source('{source}');").fetchdf()

    def select_by_date(self, date_from, date_to):
        
        return self.connection.execute(f"SELECT * FROM news_meta WHERE strptime(timestamp, '%Y-%m-%dT%H:%M:%S%z') BETWEEN strptime('{date_from}','%Y-%m-%d') AND strptime('{date_to}','%Y-%m-%d')").fetchdf()

    def select_by_date_and_term(self, date_from, date_to, term):
        print(date_from, date_to, term)
        return self.connection.execute(f"SELECT * FROM news_meta WHERE term = '{term}' AND strptime(timestamp, '%Y-%m-%dT%H:%M:%S%z') BETWEEN strptime('{date_from}','%Y-%m-%d') AND strptime('{date_to}','%Y-%m-%d')").fetchdf()

    def delete_all(self):
        self.connection.execute("DELETE FROM news_meta")

    def export(self, csv_file):
        self.connection.execute(f"EXPORT DATABASE '{csv_file}';")

    def close(self):
        self.connection.close()
        


In [4]:
# resources = {"requests_resource": 1}, num_cpus=num_cpus
@ray.remote
def try_request(url, headers, params):

    # resp_proxy = requests.get('https://free-proxy-list.net/')
    # df = pd.read_html(resp_proxy.text)[0]
    # df_http = df[df['Https']=='no']
    # df_https = df[df['Https']=='yes']
    # proxy_http = f'https://{df_http["IP Address"].values[0]}:{df_http["Port"].values[0]}'
    # proxy_https = f'https://{df_https["IP Address"].values[0]}:{df_https["Port"].values[0]}'

    proxies={
        "http": 'socks5://193.35.18.30:30808',
        "https": 'socks5://193.35.18.30:30808'
    }

    response_page = requests.request("GET", url, headers=headers, params=params, timeout=5)
    try:
        response = response_page.text

        if response_page.status_code > 300:
            print("Status code error: " + str(response_page.status_code))
            return False, params

        return True, response
    except requests.exceptions.JSONDecodeError as e:
        print(f"Bad Request: GET {url} \n Status Code: {response_page.status_code} | Error : {e}")
        return False, params
    except requests.exceptions.Timeout:
        print("Timed out")
        return False, params
    except requests.exceptions.RequestException as e:
        print(f"Bad Request: GET {url} \n")
        return False, params
    

In [27]:
def get(host, url, headers):
    connection = HTTPSConnection(host)
    connection.request('GET', url,'', headers)

    response = connection.getresponse()
    location_header = response.getheader('location')
        
    if response.status < 400:
        if location_header is None:
            return 200, response.read()
        else:
            location = urljoin(url, location_header)
            return get(host, location, headers)
    else: return response.status, None
    
async def get_async(host, path, headers):
    async with aiohttp.ClientSession(headers=headers) as session:
        async with session.post(f'https://www.ft.com{path}') as response:
            location_header = None
            
            try:    
                location_header = response.headers['location']
            except KeyError as e:
                pass
            
            if response.status < 400:

                if location_header is None:
                    return 200, await response.text()
                else:
                    location = urljoin(path, location_header)
                    return await get_async(host, location, headers)
            else: return response.status, None

In [28]:

'''
JS Fetch copied from the developer console

fetch("https://www.ft.com/search?q=Tesla&sort=date&isFirstView=true", {
  "headers": {
    "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",
    "accept-language": "en-GB,en-US;q=0.9,en;q=0.8",
    "if-none-match": "W/\"31194-I0PwYDAneq75TUQYyVdblQpxnZo\"",
    "priority": "u=0, i",
    "sec-ch-ua": "\"Not-A.Brand\";v=\"99\", \"Chromium\";v=\"124\"",
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": "\"macOS\"",
    "sec-fetch-dest": "document",
    "sec-fetch-mode": "navigate",
    "sec-fetch-site": "same-origin",
    "sec-fetch-user": "?1",
    "upgrade-insecure-requests": "1"
  },
  "referrer": "https://www.ft.com/search?q=Tesla",
  "referrerPolicy": "strict-origin-when-cross-origin",
  "body": null,
  "method": "GET",
  "mode": "cors",
  "credentials": "include"
});
'''

ft_url = 'https://www.ft.com/search'
ft_headers = {
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
    'accept-language': 'en-GB,en-US;q=0.9,en;q=0.8',
    'priority': 'u=0, i',
    'sec-ch-ua': '\"Not-A.Brand\";v=\"99\", \"Chromium\";v=\"124\"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '\"macOS\"',
    'sec-fetch-dest': 'document',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-site': 'same-origin',
    'sec-fetch-user': '?1',
    'upgrade-insecure-requests': '1'
}
items_per_page = 25

newsMetaRepository = NewsMetaRepository('meta/ft')

def get_params(search_term, page, date_from, date_to):
    return {
        'q': search_term,
        'page': page,
        'dateFrom': date_from,
        'dateTo': date_to   
    }

# resources = {"requests_resource": 1}, num_cpus=num_cpus
async def add_to_repository(meta_list, keyword, page, dates):
    reqeust_success = False
    request_retries = 0
    print(f'Fetching page {page} ...')
    
    while not reqeust_success and request_retries < 3:
        try:
            async with aiohttp.ClientSession(headers = ft_headers) as session:
                async with session.get(
                        url = ft_url,
                        params = get_params(keyword, page+1, dates['start_date'], dates['end_date'])
                ) as response:
                    soup = BeautifulSoup(await response.text(), 'html.parser')
                    search_result_list = soup.find(class_='search-results__list')
            
                    for search_result_list_item in search_result_list.children:
                        item_article_content: bs4.PageElement = search_result_list_item.find('div', class_='o-teaser__content')
            
                        if item_article_content is not None:
            
                            item_article_title_element = item_article_content.find("div", class_="o-teaser__heading").find("a", class_="js-teaser-heading-link")
            
                            item_article_title: str = (item_article_title_element.text
                                                       .replace("'","''"))
                            item_article_url = item_article_title_element['href']
                            
                            item_article_timestamp = ""
                            try:
                                item_article_timestamp = item_article_content.find("div", class_='o-teaser__timestamp').find("time")["datetime"];
                            except Exception as e:
                                pass
            
                            meta_list.append({
                                'title': item_article_title,
                                'url': item_article_url,
                                'timestamp': item_article_timestamp,
                                'term': keyword
                            })
        
                    reqeust_success = True
        
            for meta in meta_list:
                try:
                    newsMetaRepository.insert(meta)
                except Exception as e:
                    print(e.with_traceback(), meta['title'])
        except AttributeError as e:
            print(f"Failed to fetch page {page}, retrying {3-request_retries-1}")
            request_retries += 1
        except Exception as e:
            # print(e.with_traceback())
            print(f"Failed to fetch page {page}, retrying {3-request_retries-1}")
            request_retries += 1

# we need a date list because there is a limit per page so we get only a chunk between two dates
def get_date_list():
    # generate a list of date pairs
    dates_list = []

    for i in range(2014, 2025):
        dates_list.append(
            {
                'start_date':f'{i}-01-01',
                'end_date':f'{i}-06-01'
            })

        if datetime.strptime(f'{i}-06-02', '%Y-%m-%d') < datetime.now():
            dates_list.append(
                {
                    'start_date':f'{i}-06-02',
                    'end_date':f'{i+1}-01-01'
                }
            )
    return dates_list

async def query_ft_news_list(keyword):
    meta_list = []
       
    # generate a list of date pairs
    dates_list = get_date_list()
    
    for dates in dates_list:
        pre_response_ok, pre_response = ray.get(try_request.remote(
            url = ft_url,
            headers = ft_headers,
            params = get_params(keyword, 1, dates['start_date'], dates['end_date'])
        ))
    
        if pre_response_ok:
            pre_soup = BeautifulSoup(pre_response, 'html.parser')
    
            total_list_items_count_html = pre_soup.find(class_='o-teaser-collection__heading o-teaser-collection__heading--half-width').text
    
            total_list_items_count = int(total_list_items_count_html.split(" ")[-1])
            
            total_pages = int(ceil((total_list_items_count / items_per_page)))
    
            print(f'Article count / Page size: {total_list_items_count}/{total_pages}, Between dates: {dates["start_date"]},{dates["end_date"]}')
            for i in range(1,int(total_pages)):
                time.sleep(2)
                await add_to_repository(meta_list, keyword, i, dates)
    
        print(f'Metalist size: {len(meta_list)}')
        newsMetaRepository.export("meta/ft")

    # newsMetaRepository.close()


In [33]:
newsMetaRepository = NewsMetaRepository('meta/ft')

def get_meta_list(term, date_from=None, date_to=None):
    if date_from is not None and date_to is not None:
        return newsMetaRepository.select_by_date_and_term(date_from, date_to, term)
    else:
        return newsMetaRepository.select_all()

async def download_article_text(content_url, article_title, download_path):
    
    # print(f'{content_url} - {article_title} - {download_path}')
    
    article_request_headers = {
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
        'accept-language': 'en-GB,en-US;q=0.9,en;q=0.8',
        'cache-control': 'max-age=0',
        'cookie': 'FTClientSessionId=14378aca-566d-498e-ae73-ccd6bd49c1c2; o-typography-fonts-loaded=1; __exponea_etc__=92101c1c-84af-48a4-8c09-b3a17ca6a484; __exponea_time2__=-0.04106616973876953; consentUUID=23d90bf6-c3b1-4566-bdaa-52f73276429b_32; consentDate=2024-05-23T17:33:41.935Z; FTCookieConsentGDPR=true; zit.data.toexclude=0; session=eyJjc3JmU2VjcmV0IjoiVW5yczVoaHNkdUYwNTZhdERSSFFYcnRpIn0=; session.sig=9aMM9iIbsfKry_1i3UtGBYQsZmQ; optimizelyEndUserId=oeu1716486185685r0.5755696125223966; ravelinDeviceId=rjs-b04e6aec-b292-4222-848c-40914c926ca6; _sp_id.4680=2110c63c-13e8-4ce0-8650-3e418e8aba77.1716627528.1.1716627528.1716627528.0f7ec480-f856-4c10-9923-88b76d972d29; liveagent_oref=https://www.ft.com/content/135e391c-82b6-42bd-8533-a33511f9f759; liveagent_sid=00a1f8ab-4b7b-451d-ac7c-bd2f32775522; liveagent_vc=2; liveagent_ptid=00a1f8ab-4b7b-451d-ac7c-bd2f32775522; FTLogin=beta; ft_social_session_c=google; FTAllocation=c545583e-e4bf-4d86-bb87-525d1d5c6700; FTConsent=behaviouraladsOnsite%3Aon%2CcookiesOnsite%3Aoff%2CcookiesUseraccept%3Aoff%2CdemographicadsOnsite%3Aon%2CenhancementByemail%3Aoff%2CenhancementByfax%3Aoff%2CenhancementByphonecall%3Aoff%2CenhancementBypost%3Aoff%2CenhancementBysms%3Aoff%2CmarketingByemail%3Aoff%2CmarketingByfax%3Aoff%2CmarketingByphonecall%3Aoff%2CmarketingBypost%3Aoff%2CmarketingBysms%3Aoff%2CmembergetmemberByemail%3Aoff%2CpermutiveadsOnsite%3Aoff%2CpersonalisedmarketingOnsite%3Aoff%2CprogrammaticadsOnsite%3Aon%2CrecommendedcontentOnsite%3Aon; FTCookieConsentSync=false; spoor-id=clwmdq0go00003b6hecx2zbs9; _csrf=UQaR1iyibMoxTTDI8iM7acd1; FTSession_s=08VFWD7kv02G07uHUl0dXGcA0wAAAY-7A8_rw8I.MEQCIE0fc_x-acXdpnB06GhZ6zMT41phIjB4l0cBo8g561J0AiAIZGCC9Sf9gPmmL9MRhF4uehfUCjVavV-cGlNL5MDD5w; ft-access-decision-policy=SUBSCRIPTION_POLICY; _sxh=1299,1301,1302,1303,1305,1307,; _sxo={"R":0,"tP":0,"tM":0,"sP":0,"sM":0,"dP":0,"dM":0,"dS":0,"tS":0,"cPs":0,"lPs":[],"sSr":0,"sWids":[],"wN":0,"cdT":0,"F":6,"RF":2,"w":0,"SFreq":0,"last_wid":0,"bid":1036,"accNo":"","clientId":"","isEmailAud":0,"isPanelAud":0,"hDW":0,"isRegAud":0,"isExAud":0,"isDropoff":0,"devT":4,"exPW":0,"Nba":-1,"userName":"","dataLayer":"","localSt":"","emailId":"","emailTag":"","subTag":"","lVd":"","oS":"","cPu":"","pspv":0,"pslv":0,"pssSr":0,"pswN":0,"psdS":0,"pscdT":0,"RP":0,"TPrice":0,"ML":"","isReCaptchaOn":false,"reCaptchaSiteKey":"","reCaptchaSecretKey":"","extRefer":"","dM2":0,"tM2":0,"sM2":0,"RA":0,"ToBlock":-1}; FTAllocation=c545583e-e4bf-4d86-bb87-525d1d5c6700; FTClientSessionId=14378aca-566d-498e-ae73-ccd6bd49c1c2; FTCookieConsentSync=false; ft-access-decision-policy=SUBSCRIPTION_POLICY',
        'dnt': '1',
        'priority': 'u=0, i',
        'sec-ch-ua': '"Not-A.Brand";v="99", "Chromium";v="124"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"macOS"',
        'sec-fetch-dest': 'document',
        'sec-fetch-mode': 'navigate',
        'sec-fetch-site': 'none',
        'sec-fetch-user': '?1',
        'upgrade-insecure-requests': '1',
        'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36'
    }
            
    article_request_ok, article_request = get(
        host='www.ft.com',
        url=content_url,
        headers = article_request_headers, 
    )

    try:
        if article_request_ok:
            soup_article = BeautifulSoup(article_request, 'html.parser')
    
            article_site = soup_article.find('article', {"id": 'site-content'})
            article_body_paragraphs = {}
            
            # print(article_site['data-article-type'])
       
            if 'full-width-graphics' in article_site['data-article-type']:
                article_body_paragraphs = soup_article.find('article', {"class": 'n-content-body'}).findAll('p', recursive=False)
            elif article_site['data-article-type'] == 'live-blog':
                for article in article_site.findAll('article', {'class': 'x-live-blog-post'}):
                    h2 = article.find('h2', string=article_title)
                    if h2:
                        p_el = [p for p in h2.next_sibling.findAll('p')]
                        p_el = list(filter(lambda x: '<a' not in x, p_el))
                        article_body_paragraphs = p_el
                        break
                        
            article_text = "".join([p.get_text() for p in article_body_paragraphs])
            
            if len(article_body_paragraphs) == 0:
                print(f"One article has no paragraphs! @ {content_url}")

            with open(download_path, 'w') as f:
                f.write(article_text)
        else:
            print("Article request failed")
    except Exception as e:
        print(f'Error: {e} @ {content_url}')
        return
        
async def download_articles(term):
    dates = get_date_list()

    def get_file_name(id):
        title = meta_list['title'][id].replace(' ','-').replace("/","-")
        return f"{title}-{parser.parse(meta_list['timestamp'][id]).timestamp()}"
    
    for dates in dates:
        meta_list = get_meta_list(term, dates['start_date'], dates['end_date'])
        
        print(f"Number of articles between {dates['start_date']} and {dates['end_date']} : {len(meta_list)}")
        
        for i in range(0, len(meta_list))[::10]:            
            idx_end = 10 if i + 10 < len(meta_list) else (len(meta_list) % 10)
        
            print(f'Processing {i} to {i+idx_end} ...')
        
            task_list = []
            for idx in range(0,idx_end):
                task_list.append(download_article_text(
                    meta_list['url'][idx + i], 
                    meta_list['title'][idx + i], 
                    f'articles/ft/{get_file_name(idx + i)}.txt')
                )
            
            await asyncio.gather(*task_list)
            sleep(0.5)


In [34]:

async def load_articles_for_company(company):
    await query_ft_news_list(company)
    download_articles(company)

# load_articles_for_company('Tesla')

await download_articles("Tesla")

2014-01-01 2014-06-01 Tesla
Number of articles between 2014-01-01 and 2014-06-01 : 53
Processing 0 to 10 ...
Processing 10 to 20 ...
Processing 20 to 30 ...
Processing 30 to 40 ...
Processing 40 to 50 ...
Processing 50 to 53 ...
2014-06-02 2015-01-01 Tesla
Number of articles between 2014-06-02 and 2015-01-01 : 75
Processing 0 to 10 ...
Error: 'NoneType' object is not subscriptable @ /video/81aca612-3991-3291-98a5-0f024db14a93
Processing 10 to 20 ...
Processing 20 to 30 ...
Processing 30 to 40 ...
Processing 40 to 50 ...
Processing 50 to 60 ...
Processing 60 to 70 ...
Processing 70 to 75 ...
2015-01-01 2015-06-01 Tesla
Number of articles between 2015-01-01 and 2015-06-01 : 78
Processing 0 to 10 ...
Processing 10 to 20 ...
Error: 'NoneType' object is not subscriptable @ /video/2cffdf8a-44cf-31e9-a261-c40cb2aec50b
Processing 20 to 30 ...
Processing 30 to 40 ...
Processing 40 to 50 ...
Processing 50 to 60 ...
Processing 60 to 70 ...
Processing 70 to 78 ...
2015-06-02 2016-01-01 Tesla
Numbe